# Calculate robusteness from pi control runs

In [274]:
# Some definitions

# directory of data
#fdir = '~/Google Drive/professional/research/FARALLON_INSTITUTE_PROJECTS/2020 NOAA MAPP/Climate_extremes_sharedfigsandcode/data/FOR SCATTER PLOT/'
fdir = '/Volumes/GoogleDrive/My Drive/Climate_extremes_sharedfigsandcode/data/Annual_TimeSeries/'

In [275]:
# Modules
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore') #filter some warning messages
import seaborn as sns

In [276]:
#detrend function: detrend use polynomial fit
def detrend(x,y,degree):
    import statsmodels.formula.api as smf
    import numpy as np
    import pandas as pd
    

    df = pd.DataFrame(columns=['y', 'x'])
    df['x'] = x
    df['y'] = pd.Series(y)   #y

    idx = np.isfinite(x) & np.isfinite(y)
    weights = np.polyfit(x[idx], y[idx], degree)
    model = np.poly1d(weights)
    results = smf.ols(formula='y ~ model(x)', data=df).fit()

    p_value=results.f_pvalue
    r2=results.rsquared_adj

    detrend_ts=np.zeros(shape=(len(y),))
    detrend_ts[:] = np.nan
    detrend_ts[idx]=y[idx]-results.fittedvalues

    #plt.figure(figsize=(10,4))
    #plt.plot(x,y)
    #plt.plot(x[idx], results.fittedvalues, 'r')
    #plt.figure(figsize=(10,4))
    #plt.plot(x, detrend_ts)
    #plt.show()

    return p_value, r2, detrend_ts

In [277]:
def num_coevents(y1, y2, pct1, pct2, runs='picontrol',loc='cclme'):
    # 
    fin1 = fdir+'original ts/'+runs+'_'+loc+'_sst.nc'
    fin2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fin1)
    modsst.close()
    modsm2 = xr.open_dataset(fin2)
    modsm2.close()
    
    models = modsm2.model

    # selec periood
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()
    mhw = list()
    drt = list()    
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1_0= modsst.sel(model=i).sst.values
        tmp2_0= modsm2.sel(model=i).sm.values
        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        
        # calculate threshold
        mhw_thr = np.nanpercentile(tmp1,pct1)
        drg_thr = np.nanpercentile(tmp2,pct2)
    
        a1 = tmp1>=mhw_thr
        a2 = tmp2<=drg_thr
        
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
        
        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
        
        nev.append(len(tmp)/30*100) #/(y2-y1+1)) #freq.
        mhw.append(len(tmp1)/30*100)  #mhw_thr)#
        drt.append(len(tmp2)/30*100)  #drg_thr)#   
    return nev , mhw, drt # sum(nev)

In [278]:
ny = 30
prc1 = 90
prc2 = 10
loc='goa'

pinev=list()
pim=list()
pid=list()
for i in range(int(500/ny)):
    nev,m0,d0 = num_coevents(i*ny+1, (i+1)*ny, prc1 , prc2,loc=loc)
    pinev.append(nev)
    pim.append(m0)
    pid.append(d0)
pinev,np.mean(pinev) ,np.mean(pim), np.mean(pid) 

([[3.3333333333333335,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   6.666666666666667,
   0.0,
   0.0],
  [0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   6.666666666666667,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   0.0],
  [0.0,
   0.0,
   6.666666666666667,
   0.0,
   6.666666666666667,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   3.3333333333333335,
   3.3333333333333335,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0],
  [3.3333333333333335,
   0.0,
   6.666666666666667,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   0.0,
   3.3333333333333335,
   0.0,
   0.0,
   3.3333333333333

In [279]:
#model by model

In [280]:
model_mean=np.mean(pinev,axis=0)
model_spread=np.std(pinev,axis=0)
model_mean

array([0.83333333, 1.25      , 1.45833333, 0.83333333, 1.25      ,
       1.66666667, 1.25      , 1.25      , 1.04166667, 0.625     ,
       1.66666667, 1.45833333, 1.04166667, 1.25      , 0.83333333,
       1.04166667, 1.45833333, 0.83333333, 0.83333333, 1.04166667,
       0.625     , 0.83333333])

In [281]:
#16 30-yr periods MMEs

In [282]:
mme_mean=np.mean(pinev,axis=1)
mme_spread=np.std(pinev,axis=1)
#internal variability mean and spread
mme_mean,np.mean(mme_spread)

(array([1.51515152, 0.75757576, 1.06060606, 1.66666667, 1.21212121,
        1.06060606, 0.60606061, 0.75757576, 0.90909091, 0.75757576,
        1.06060606, 1.21212121, 1.21212121, 1.06060606, 1.51515152,
        1.36363636]),
 1.7494140983427964)

In [283]:
#MME mean and spread

In [284]:
mme_model_mean=np.mean(mme_mean)
mme_model_spread=np.std(mme_mean)
mme_model_mean,mme_model_spread

(1.1079545454545454, 0.29690180694300916)

In [285]:
#cross model mean and spread

In [286]:
cross_model_mean=np.mean(model_mean)
cross_model_spread=np.std(model_mean)
cross_model_mean,cross_model_spread

(1.1079545454545452, 0.30376922220351205)

In [287]:
## Calculate thresholds for PI Control runs

In [288]:
def get_thre(loc,y1,y2,prc1,prc2):
    # open data: anomalies (without trend)

    modsst = xr.open_dataset(fdir+'original ts/picontrol_'+loc+'_sst.nc')
    modsst.close()
    modsm2 = xr.open_dataset(fdir+'original ts/picontrol_sm.nc')
    modsm2.close()

    # select period 
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))
    mhw_thr = list()
    drg_thr = list()
    models = list()

    # Calculate thresholds for each model
    for ix,i in enumerate(modsm2.model.values):
        tmp1_0= modsst.sel(model=i).sst.values
        tmp2_0= modsm2.sel(model=i).sm.values
        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        
        mhw_thr.append(np.nanpercentile(tmp1,prc1))
        drg_thr.append(np.nanpercentile(tmp2,prc2))

        models.append(i)
    
    return mhw_thr, drg_thr , models

In [289]:
# calculate number of events for control, historical and future
# based on pi control thresholds

def freq_coevents(fdir, runs, loc, mhw_thr, drg_thr, models, y1, y2):
    #pi control
    fi1 = fdir+'original ts/'+runs+'_'+loc+'_sst.nc'
    fi2 = fdir+'original ts/'+runs+'_sm.nc'
    modsst = xr.open_dataset(fi1)
    modsst.close()
    modsm2 = xr.open_dataset(fi2)
    modsm2.close()
    models=modsm2.model

    # select period
    modsst = modsst.sel(year=slice(y1,y2))
    modsm2 = modsm2.sel(year=slice(y1,y2))

    nev = list()
    drt=list()
    mhw=list()
    for ix,i in enumerate(models):
        #print(ix,i)
        tmp1_0= modsst.sel(model=i).sst.values
        tmp2_0= modsm2.sel(model=i).sm.values
        
        #detrend
        nsample = np.linspace(1,len(tmp1_0), len(tmp1_0))
        [p_value, r2, tmp1]= detrend(nsample,tmp1_0,1)
        nsample = np.linspace(1,len(tmp2_0), len(tmp2_0))
        [p_value, r2, tmp2]= detrend(nsample,tmp2_0,1)
        
        a1 = tmp1>=mhw_thr[ix]
        a2 = tmp2<=drg_thr[ix]
        tmp = np.full((len(modsst.year.values),1),1)
        tmp = tmp[a1*a2]
        
        tmp1 = np.full((len(modsst.year.values),1),1)
        tmp1 = tmp1[a1]
        tmp2 = np.full((len(modsst.year.values),1),1)
        tmp2 = tmp2[a2]
        
        nev.append(len(tmp)/30*100)
        mhw.append(len(tmp1)/30*100)
        drt.append(len(tmp2)/30*100)
    
    freq = np.array(nev) #/(y2-y1+1)
    
    return nev,  mhw, drt #mean no. of events
#freq, np.round(np.nanmean(freq),4), np.round(np.nanstd(freq),4) #mean frequency


In [290]:
def freq_difruns(loc,ny, y0,y,prc1, prc2):
    
    # calculate threshold in pi control
    #y2=2014
    #y1=y2-ny+1
    mhw_thr, drg_thr, models = get_thre(loc,y0,y,prc1,prc2)

    print ('Percentiles: '+str(prc1)+'/'+str(prc2))
    # pi control
    print('piControl: '+str(y0)+'-'+str(y))
    #y2=2014
    #y1=y2-ny+1
    freq0, mfrq, sfrq = freq_coevents(fdir,'picontrol',loc, mhw_thr,drg_thr, models,y0,y)
    print(mfrq,sfrq)

    # historical 1
    y1 = 1900
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # historical 3
    y1 = 1950
    y2 = y1+ny-1
    print('\nhistorical: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)


    # historical 2
    y2=2014
    y1=y2-ny+1
    print('\nhistorical 2: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'historical',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future 1
    y2=2099
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq2, mfrq2, sfrq2 = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq2,sfrq2)
    
    # future 2
    y2=2069
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq, mfrq, sfrq = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq,sfrq)

    # future 2
    y2=2044
    y1=y2-ny+1
    print('\nfuture: '+str(y1)+'-'+str(y2))
    freq1, mfrq1, sfrq1 = freq_coevents(fdir,'ssp585',loc,mhw_thr,drg_thr, models, y1,y2)
    print(mfrq1,sfrq1)
    
    return freq0,freq1,freq2,mfrq1, sfrq1,mfrq2, sfrq2

In [291]:
# Frequency of events - detrended data, using pi control threshols
## percentiles 90-10

In [292]:

print ('\nPercentiles: '+str(prc1)+'/'+str(prc2))
print('\nProbable # events in '+str(ny)+' years'+': '+str(ny*0.01))
funev=list()
hinev=list()
test=list()
him=list()
hid=list()
fum=list()
fud=list()
for i in range(int(500/ny)):
    pi,hi,fu,m1,d1,m2,d2 = freq_difruns(loc,ny,i*ny+1, (i+1)*ny, prc1, prc2)
    test.append(pi)
    funev.append(fu)
    hinev.append(hi)
    him.append(m1)
    hid.append(d1)
    fum.append(m2)
    fud.append(d2)
np.mean(funev),np.mean(hinev),np.mean(him),np.mean(hid),np.mean(fum),np.mean(fud)


Percentiles: 90/10

Probable # events in 30 years: 0.3
Percentiles: 90/10
piControl: 1-30
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1900-1929
[13.333333333333334, 20.0, 6.666666666666667, 16.666666666666664, 10.0, 3.3333333333333335, 10.0, 10.0, 20.0, 30.0, 16.666666666666664, 6.666666666666667, 16.666666666666664, 0.0, 10.0, 10.0, 10.0, 13.333333333333334, 16.666666666666664, 10.0, 16.666666666666664, 6.666666666666667] [13.333333333333334, 6.666666666666667, 13.333333333333334, 10.0, 26.666666666666668, 33.33333333333333, 13.333333333333334, 16.666666666666664, 16.666666666666664, 0.0, 23.333333333333332, 3.3333333333333335, 23.333333333333332, 20.0, 3.3333333333333335, 10.0, 6.666666666666667, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1950-1979
[13

Percentiles: 90/10
piControl: 61-90
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1900-1929
[13.333333333333334, 13.333333333333334, 20.0, 20.0, 23.333333333333332, 3.3333333333333335, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 13.333333333333334, 20.0, 6.666666666666667, 13.333333333333334, 0.0, 3.3333333333333335, 3.3333333333333335, 10.0, 20.0, 13.333333333333334, 13.333333333333334, 20.0, 13.333333333333334] [6.666666666666667, 16.666666666666664, 10.0, 6.666666666666667, 3.3333333333333335, 26.666666666666668, 10.0, 16.666666666666664, 16.666666666666664, 6.666666666666667, 26.666666666666668, 13.333333333333334, 16.666666666666664, 10.0, 3.3333333333333335, 26.666666666666668, 6.666666666666667, 10.0, 3.3333333333333335, 13.33333333333

[10.0, 3.3333333333333335, 0.0, 16.666666666666664, 13.333333333333334, 6.666666666666667, 10.0, 13.333333333333334, 26.666666666666668, 16.666666666666664, 3.3333333333333335, 3.3333333333333335, 13.333333333333334, 6.666666666666667, 6.666666666666667, 13.333333333333334, 13.333333333333334, 0.0, 10.0, 0.0, 6.666666666666667, 13.333333333333334] [3.3333333333333335, 26.666666666666668, 10.0, 6.666666666666667, 10.0, 3.3333333333333335, 16.666666666666664, 13.333333333333334, 33.33333333333333, 20.0, 10.0, 16.666666666666664, 16.666666666666664, 13.333333333333334, 16.666666666666664, 6.666666666666667, 0.0, 26.666666666666668, 6.666666666666667, 13.333333333333334, 10.0, 10.0]
Percentiles: 90/10
piControl: 121-150
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

histori

[6.666666666666667, 3.3333333333333335, 6.666666666666667, 10.0, 16.666666666666664, 6.666666666666667, 6.666666666666667, 16.666666666666664, 13.333333333333334, 10.0, 10.0, 3.3333333333333335, 20.0, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 10.0, 30.0, 23.333333333333332, 0.0, 6.666666666666667, 10.0] [3.3333333333333335, 30.0, 16.666666666666664, 10.0, 6.666666666666667, 3.3333333333333335, 6.666666666666667, 23.333333333333332, 30.0, 6.666666666666667, 3.3333333333333335, 3.3333333333333335, 6.666666666666667, 10.0, 16.666666666666664, 0.0, 10.0, 20.0, 3.3333333333333335, 10.0, 20.0, 10.0]
Percentiles: 90/10
piControl: 181-210
[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0] [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0]

historical: 1900-1929
[10.0, 26.666666666666668, 6.666666666666667, 16.666666

[23.333333333333332, 6.666666666666667, 23.333333333333332, 20.0, 23.333333333333332, 23.333333333333332, 16.666666666666664, 23.333333333333332, 6.666666666666667, 3.3333333333333335, 6.666666666666667, 13.333333333333334, 10.0, 13.333333333333334, 23.333333333333332, 16.666666666666664, 13.333333333333334, 13.333333333333334, 13.333333333333334, 16.666666666666664, 6.666666666666667, 16.666666666666664] [10.0, 13.333333333333334, 13.333333333333334, 16.666666666666664, 16.666666666666664, 30.0, 13.333333333333334, 20.0, 23.333333333333332, 30.0, 20.0, 30.0, 16.666666666666664, 6.666666666666667, 20.0, 6.666666666666667, 10.0, 6.666666666666667, 6.666666666666667, 23.333333333333332, 23.333333333333332, 13.333333333333334]

future: 2016-2045
[16.666666666666664, 0.0, 20.0, 10.0, 16.666666666666664, 20.0, 6.666666666666667, 13.333333333333334, 10.0, 6.666666666666667, 3.3333333333333335, 6.666666666666667, 13.333333333333334, 6.666666666666667, 13.333333333333334, 13.333333333333334, 1

[10.0, 20.0, 23.333333333333332, 20.0, 10.0, 3.3333333333333335, 16.666666666666664, 6.666666666666667, 6.666666666666667, 16.666666666666664, 3.3333333333333335, 10.0, 16.666666666666664, 23.333333333333332, 16.666666666666664, 6.666666666666667, 3.3333333333333335, 0.0, 16.666666666666664, 13.333333333333334, 6.666666666666667, 3.3333333333333335] [16.666666666666664, 13.333333333333334, 10.0, 10.0, 16.666666666666664, 13.333333333333334, 0.0, 6.666666666666667, 30.0, 13.333333333333334, 16.666666666666664, 3.3333333333333335, 13.333333333333334, 16.666666666666664, 10.0, 3.3333333333333335, 16.666666666666664, 13.333333333333334, 0.0, 10.0, 16.666666666666664, 16.666666666666664]

future: 2040-2069
[16.666666666666664, 16.666666666666664, 20.0, 23.333333333333332, 23.333333333333332, 10.0, 16.666666666666664, 13.333333333333334, 6.666666666666667, 23.333333333333332, 6.666666666666667, 6.666666666666667, 20.0, 13.333333333333334, 6.666666666666667, 6.666666666666667, 10.0, 10.0, 13.

[20.0, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 6.666666666666667, 16.666666666666664, 20.0, 3.3333333333333335, 16.666666666666664, 0.0, 6.666666666666667, 13.333333333333334, 6.666666666666667, 10.0, 20.0, 6.666666666666667, 23.333333333333332, 3.3333333333333335, 6.666666666666667, 23.333333333333332, 16.666666666666664, 3.3333333333333335] [6.666666666666667, 23.333333333333332, 23.333333333333332, 16.666666666666664, 13.333333333333334, 10.0, 26.666666666666668, 16.666666666666664, 20.0, 3.3333333333333335, 3.3333333333333335, 3.3333333333333335, 10.0, 23.333333333333332, 23.333333333333332, 16.666666666666664, 10.0, 13.333333333333334, 10.0, 6.666666666666667, 23.333333333333332, 20.0]

future: 2070-2099
[13.333333333333334, 13.333333333333334, 16.666666666666664, 6.666666666666667, 10.0, 13.333333333333334, 16.666666666666664, 3.3333333333333335, 6.666666666666667, 6.666666666666667, 3.3333333333333335, 10.0, 13.333333333333334, 20.0, 16.666666666666664, 16.666

[0.0, 0.0, 10.0, 13.333333333333334, 13.333333333333334, 20.0, 23.333333333333332, 13.333333333333334, 6.666666666666667, 3.3333333333333335, 0.0, 10.0, 6.666666666666667, 13.333333333333334, 16.666666666666664, 10.0, 10.0, 0.0, 10.0, 20.0, 20.0, 33.33333333333333] [6.666666666666667, 3.3333333333333335, 3.3333333333333335, 16.666666666666664, 6.666666666666667, 13.333333333333334, 6.666666666666667, 20.0, 3.3333333333333335, 20.0, 6.666666666666667, 6.666666666666667, 26.666666666666668, 6.666666666666667, 16.666666666666664, 6.666666666666667, 6.666666666666667, 3.3333333333333335, 13.333333333333334, 13.333333333333334, 30.0, 6.666666666666667]

historical 2: 1985-2014
[10.0, 6.666666666666667, 10.0, 16.666666666666664, 6.666666666666667, 16.666666666666664, 16.666666666666664, 10.0, 16.666666666666664, 0.0, 6.666666666666667, 13.333333333333334, 0.0, 10.0, 30.0, 6.666666666666667, 13.333333333333334, 13.333333333333334, 10.0, 16.666666666666664, 16.666666666666664, 26.6666666666666

[6.666666666666667, 0.0, 23.333333333333332, 13.333333333333334, 23.333333333333332, 13.333333333333334, 6.666666666666667, 13.333333333333334, 6.666666666666667, 3.3333333333333335, 10.0, 10.0, 20.0, 13.333333333333334, 10.0, 3.3333333333333335, 20.0, 3.3333333333333335, 0.0, 23.333333333333332, 0.0, 33.33333333333333] [13.333333333333334, 6.666666666666667, 0.0, 20.0, 6.666666666666667, 13.333333333333334, 10.0, 26.666666666666668, 26.666666666666668, 6.666666666666667, 6.666666666666667, 30.0, 20.0, 10.0, 0.0, 20.0, 6.666666666666667, 10.0, 20.0, 0.0, 20.0, 6.666666666666667]

historical 2: 1985-2014
[16.666666666666664, 3.3333333333333335, 16.666666666666664, 16.666666666666664, 6.666666666666667, 3.3333333333333335, 0.0, 3.3333333333333335, 16.666666666666664, 0.0, 16.666666666666664, 13.333333333333334, 16.666666666666664, 20.0, 30.0, 6.666666666666667, 23.333333333333332, 16.666666666666664, 6.666666666666667, 23.333333333333332, 10.0, 26.666666666666668] [23.333333333333332, 16

(2.0928030303030307,
 1.8655303030303032,
 10.359848484848484,
 12.698863636363637,
 11.145833333333332,
 13.001893939393938)

In [293]:
loc

'goa'

In [294]:
np.mean(funev)

2.0928030303030307

In [295]:
np.mean(hinev)

1.8655303030303032

In [296]:
np.mean(test),np.mean(pinev)

(1.1079545454545454, 1.1079545454545454)

In [297]:
#model by model

In [298]:
fu_model_mean=np.mean(funev,axis=0)
fu_model_spread=np.std(funev,axis=0)
fu_model_mean

array([1.04166667, 0.20833333, 5.83333333, 0.        , 0.41666667,
       1.04166667, 1.45833333, 3.95833333, 3.125     , 1.25      ,
       0.20833333, 5.41666667, 1.25      , 4.16666667, 1.04166667,
       4.58333333, 0.        , 0.41666667, 1.04166667, 2.70833333,
       5.41666667, 1.45833333])

In [299]:
np.mean(fu_model_mean),np.std(fu_model_mean)

(2.0928030303030303, 1.9026551348610288)

In [300]:
#mme mean and spread

In [301]:
fu_mme_mean=np.mean(funev,axis=1)
fu_mme_spread=np.std(funev,axis=1)
np.mean(fu_mme_spread)

2.6541116237218976

In [302]:
fu_mme_mean,np.mean(fu_mme_mean),np.std(fu_mme_mean)

(array([2.42424242, 1.51515152, 2.27272727, 1.96969697, 1.81818182,
        1.81818182, 2.42424242, 2.87878788, 2.72727273, 1.66666667,
        1.81818182, 2.12121212, 2.12121212, 1.81818182, 2.42424242,
        1.66666667]),
 2.0928030303030307,
 0.3871010120448806)

In [303]:
hi_m=np.mean(him,axis=1)
fu_m=np.mean(fum,axis=1)
diff=list()
zip_obj=zip(fu_m,hi_m)
for i , j in zip_obj:
        diff.append(i-j)
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.7859848484848486, 0.8039209433455996)

In [304]:
hi_d=np.mean(hid,axis=1)
fu_d=np.mean(fud,axis=1)
diff=list()
zip_obj=zip(fu_d,hi_d)
for i , j in zip_obj:
        diff.append(i-j)
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.30303030303030365, 1.0686921196716574)

In [305]:
hi_mme_mean=np.mean(hinev,axis=1)
diff=list()
zip_obj=zip(fu_mme_mean,hi_mme_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.22727272727272743, 0.43519413988924466)

In [306]:
#16 MMEs diff and spread

In [307]:
diff=list()
zip_obj=zip(fu_mme_mean,mme_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[0.9090909090909094,
 0.7575757575757577,
 1.2121212121212122,
 0.30303030303030276,
 0.6060606060606066,
 0.7575757575757576,
 1.8181818181818188,
 2.1212121212121215,
 1.8181818181818183,
 0.9090909090909093,
 0.757575757575758,
 0.9090909090909094,
 0.9090909090909094,
 0.757575757575758,
 0.9090909090909094,
 0.3030303030303034]

In [308]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.9848484848484851, 0.5025189076296062)

In [309]:
#model by model diff and spread

In [310]:
diff=list()
zip_obj=zip(fu_model_mean,model_mean)
for i , j in zip_obj:
        diff.append(i-j)
diff

[0.20833333333333337,
 -1.0416666666666667,
 4.375000000000002,
 -0.8333333333333334,
 -0.8333333333333333,
 -0.6249999999999998,
 0.20833333333333326,
 2.7083333333333326,
 2.083333333333334,
 0.625,
 -1.4583333333333335,
 3.958333333333335,
 0.20833333333333326,
 2.916666666666667,
 0.20833333333333337,
 3.541666666666666,
 -1.4583333333333333,
 -0.4166666666666667,
 0.20833333333333337,
 1.6666666666666667,
 4.791666666666666,
 0.6249999999999999]

In [311]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(0.9848484848484851, 1.9095943900784051)

In [312]:
np.mean(fum,axis=0)
np.mean(fum,axis=0)
fum

[[10.0,
  16.666666666666664,
  23.333333333333332,
  6.666666666666667,
  6.666666666666667,
  0.0,
  6.666666666666667,
  10.0,
  6.666666666666667,
  16.666666666666664,
  3.3333333333333335,
  10.0,
  16.666666666666664,
  10.0,
  23.333333333333332,
  20.0,
  10.0,
  3.3333333333333335,
  20.0,
  13.333333333333334,
  10.0,
  3.3333333333333335],
 [0.0,
  13.333333333333334,
  23.333333333333332,
  6.666666666666667,
  10.0,
  0.0,
  16.666666666666664,
  3.3333333333333335,
  6.666666666666667,
  16.666666666666664,
  3.3333333333333335,
  10.0,
  13.333333333333334,
  10.0,
  16.666666666666664,
  16.666666666666664,
  3.3333333333333335,
  0.0,
  20.0,
  6.666666666666667,
  6.666666666666667,
  10.0],
 [0.0,
  3.3333333333333335,
  26.666666666666668,
  13.333333333333334,
  10.0,
  0.0,
  3.3333333333333335,
  3.3333333333333335,
  6.666666666666667,
  6.666666666666667,
  6.666666666666667,
  10.0,
  10.0,
  20.0,
  16.666666666666664,
  10.0,
  16.666666666666664,
  10.0,
 

In [313]:
diff=list()
zip_obj=zip(np.mean(fum,axis=1),np.mean(pim,axis=1))
for i , j in zip_obj:
        diff.append(i-j)
diff

[1.2121212121212128,
 -0.30303030303030454,
 0.9090909090909083,
 -0.7575757575757596,
 0.45454545454545503,
 0.7575757575757578,
 1.6666666666666679,
 2.272727272727275,
 3.333333333333332,
 1.5151515151515156,
 1.6666666666666679,
 0.6060606060606037,
 1.8181818181818183,
 1.0606060606060606,
 1.5151515151515156,
 0.6060606060606055]

In [314]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(1.1458333333333333, 0.9461644303433822)

In [315]:
diff_mean=np.mean(diff)
diff_spread=np.std(diff)
diff_mean,diff_spread

(1.1458333333333333, 0.9461644303433822)

In [316]:
pim[0]

[10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0,
 10.0]

In [317]:
pim[0],np.mean(pim[0])

([10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0,
  10.0],
 10.0)

In [318]:
fum[0],np.mean(fum[0])

([10.0,
  16.666666666666664,
  23.333333333333332,
  6.666666666666667,
  6.666666666666667,
  0.0,
  6.666666666666667,
  10.0,
  6.666666666666667,
  16.666666666666664,
  3.3333333333333335,
  10.0,
  16.666666666666664,
  10.0,
  23.333333333333332,
  20.0,
  10.0,
  3.3333333333333335,
  20.0,
  13.333333333333334,
  10.0,
  3.3333333333333335],
 11.212121212121213)

In [319]:
#df= pd.DataFrame({'pi_count':pim[0],'fu_count': fum[0]})
#out_path = "./check_thr1.xlsx"
#writer = pd.ExcelWriter(out_path, engine='xlsxwriter')
#df.to_excel(writer, sheet_name='Sheet1')

# Close the Pandas Excel writer and output the Excel file.
#writer.save()